### Ingest qualifying json files

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from formula1.formula1_constants import raw_folder_path, processed_folder_path, presentation_folder_path

In [ ]:
from formula1.formula1_utils import add_ingestion_date, re_arrange_partition_column, df_column_to_list

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
qualifying_schema = StructType(fields=[StructField("qualifyId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), True),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("constructorId", IntegerType(), True),
                                      StructField("number", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("q1", StringType(), True),
                                      StructField("q2", StringType(), True),
                                      StructField("q3", StringType(), True),
                                     ])

In [ ]:
qualifying_df = spark.read \
.schema(qualifying_schema) \
.option("multiLine", True) \
.json(f"{raw_folder_path}/qualifying")

##### Step 2 - Rename columns and add new columns
1. Rename qualifyingId, driverId, constructorId and raceId
1. Add ingestion_date with current timestamp

In [ ]:
qualifying_with_ingestion_date_df = add_ingestion_date(qualifying_df)

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
final_df = qualifying_with_ingestion_date_df.withColumnRenamed("qualifyId", "qualify_id") \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumnRenamed("constructorId", "constructor_id") \
.withColumn("data_source", lit(v_data_source))

##### Step 3 - Write to output to processed container in parquet format

In [ ]:
final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/qualifying")